In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in the words
words = open('names.txt','r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
# print(itos)
print(vocab_size)

27


In [4]:

# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
    X, Y = [], []
  
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [5]:
#utility function we will use later when comparing manual gradients to Pytorch gradients
def cmp(s,dt,t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt-t.grad).abs().max().item()
    print(f'{s:15s} | exact :{str(ex):5s} | approximate:{str(app):5s} | maxdiff:{maxdiff}')

In [6]:
# MLP revisited
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64
# the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)

W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5) #* 0.2
print(W1.shape)
b1 = torch.randn(n_hidden,                        generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.01
b2 = torch.randn(vocab_size,                      generator=g) * 0

bngain = torch.ones((1,n_hidden))*0.1+1.0
bnbias = torch.zeros((1,n_hidden))*0.1
# bnmean_running = torch.zeros((1, n_hidden))
# bnstd_running = torch.zeros((1, n_hidden))

parameters = [C, W1,b1, W2, b2 , bngain , bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

torch.Size([30, 64])
4137


In [7]:
print(W1[0].shape)

torch.Size([64])


In [8]:
batch_size = 32
n = batch_size
# Construct a mini batch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb,Yb = Xtr[ix], Ytr[ix] # batch X , Y

In [9]:
Xb.shape

torch.Size([32, 3])

In [10]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

4137


In [11]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
print(logits.shape)


# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values

# print("logit_maxees" , logit_maxes)

norm_logits = logits - logit_maxes # subtract max for numerical stability
print(norm_logits.shape, "norm")
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)

print(counts_sum.shape , "counts_sum")

counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv

print("counts_sum_inv" , counts_sum_inv.shape)

logprobs = probs.log()
print(logprobs.shape, "Logprobs")
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

torch.Size([32, 27])
torch.Size([32, 27]) norm
torch.Size([32, 1]) counts_sum
counts_sum_inv torch.Size([32, 1])
torch.Size([32, 27]) Logprobs


tensor(3.3442, grad_fn=<NegBackward0>)

In [40]:
print(Yb)

tensor([ 1, 12,  0,  5,  9, 18,  9, 16,  1,  0,  0,  1,  0,  0, 14, 12,  0,  0,
         0,  8, 25,  5,  0, 20, 19, 15, 12, 22, 22,  2, 21, 18])


In [44]:
# Backpropagation through exactly all of the variables

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n),Yb] = -1.0/n

# print(dlogprobs , "dlogprobs")

dprobs = (1.0/probs) * dlogprobs
dcounts_sum_inv = (counts*dprobs).sum(1, keepdim=True)
dcounts = counts_sum_inv*dprobs
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts)*dcounts_sum
dnorm_logits = counts * dcounts
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
dlogits += F.one_hot(logits.max(1).indices, num_classes = logits.shape[1])* dlogit_maxes
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1.0 - h**2) * dh
dbngain = (bnraw*dhpreact).sum(0 , keepdim=True)
dbnraw = bngain*dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5)*dbnvar_inv
dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
dbndiff+= (2*bndiff)*dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn+= 1.0/n * (torch.ones_like(hprebn)*dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
# print(dhprebn.shape)
print(dC.shape , "shape")


cmp('logprobs', dlogprobs, logprobs)
cmp('probs' , dprobs, probs)
cmp('counts_sum_inv' , dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum' , dcounts_sum, counts_sum)
cmp('counts' , dcounts, counts)
cmp('norm_logits' , dnorm_logits, norm_logits)
cmp('logit_maxes' , dlogit_maxes, logit_maxes)
cmp('dlogits' , dlogits, logits)
cmp('h' , dh, h)
cmp('W2' , dW2, W2)
cmp('b2' , db2, b2)
cmp('hpreact' , dhpreact, hpreact)

cmp('bngain' , dbngain, bngain)
cmp('bnbias' , dbnbias,bnbias)
cmp('bnraw' , dbnraw, bnraw)

cmp('bndiff' ,dbndiff, bndiff )
cmp('bnvar_inv' ,dbnvar_inv, bnvar_inv)
cmp('bnvar' ,dbnvar, bnvar )
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

torch.Size([27, 10]) shape
logprobs        | exact :True  | approximate:True  | maxdiff:0.0
probs           | exact :True  | approximate:True  | maxdiff:0.0
counts_sum_inv  | exact :True  | approximate:True  | maxdiff:0.0
counts_sum      | exact :True  | approximate:True  | maxdiff:0.0
counts          | exact :True  | approximate:True  | maxdiff:0.0
norm_logits     | exact :True  | approximate:True  | maxdiff:0.0
logit_maxes     | exact :True  | approximate:True  | maxdiff:0.0
dlogits         | exact :True  | approximate:True  | maxdiff:0.0
h               | exact :True  | approximate:True  | maxdiff:0.0
W2              | exact :True  | approximate:True  | maxdiff:0.0
b2              | exact :True  | approximate:True  | maxdiff:0.0
hpreact         | exact :True  | approximate:True  | maxdiff:0.0
bngain          | exact :True  | approximate:True  | maxdiff:0.0
bnbias          | exact :True  | approximate:True  | maxdiff:0.0
bnraw           | exact :True  | approximate:True  | maxdiff:0.

In [42]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.7919578552246094 diff: 0.0


In [43]:
# backward pass

dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact :False | approximate:True  | maxdiff:5.122274160385132e-09


In [16]:
F.softmax(logits, 1)[0]
dlogits[0] * n
dlogits[0].sum()

tensor(-2.3283e-09, grad_fn=<SumBackward0>)

In [17]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [18]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact :False | approximate:True  | maxdiff:9.313225746154785e-10


In [19]:
dhprebn.shape, bngain.shape, bnvar_inv.shape, dbnraw.shape, dbnraw.sum(0).shape

(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 torch.Size([64]))

In [46]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
#     bnmean = hprebn.mean(0, keepdim=True)
#     bnvar = hprebn.var(0, keepdim=True, unbiased=True)
#     bnvar_inv = (bnvar + 1e-5)**-0.5
#     bnraw = (hprebn - bnmean) * bnvar_inv
#     hpreact = bngain * bnraw + bnbias

    bnmeani = 1/n*hprebn.sum(0, keepdim=True)
    bndiff = hprebn - bnmeani
    bndiff2 = bndiff**2
    bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = bndiff * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    #     loss = F.cross_entropy(logits, Yb) # loss function
    logit_maxes = logits.max(1, keepdim=True).values

    # print("logit_maxees" , logit_maxes)
    norm_logits = logits - logit_maxes # subtract max for numerical stability
    counts = norm_logits.exp()
    counts_sum = counts.sum(1, keepdims=True)
    counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
    probs = counts * counts_sum_inv
    logprobs = probs.log()
#     print(logprobs.shape, "Logprobs")
    loss = -logprobs[range(n), Yb].mean()
    
    # backward pass
    for p in parameters:
        p.grad = None
    #loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    
    dlogprobs = torch.zeros_like(logprobs)
    dlogprobs[range(n),Yb] = -1.0/n

    # print(dlogprobs , "dlogprobs")
    
    dprobs = (1.0/probs) * dlogprobs
    dcounts_sum_inv = (counts*dprobs).sum(1, keepdim=True)
    dcounts = counts_sum_inv*dprobs
    dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
    dcounts += torch.ones_like(counts)*dcounts_sum
    dnorm_logits = counts * dcounts
    dlogits = dnorm_logits.clone()
    dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
    dlogits += F.one_hot(logits.max(1).indices, num_classes = logits.shape[1])* dlogit_maxes
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    dhpreact = (1.0 - h**2) * dh
    dbngain = (bnraw*dhpreact).sum(0 , keepdim=True)
    dbnraw = bngain*dhpreact
    dbnbias = dhpreact.sum(0, keepdim=True)
    dbndiff = bnvar_inv * dbnraw
    dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
    dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5)*dbnvar_inv
    dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
    dbndiff+= (2*bndiff)*dbndiff2
    dhprebn = dbndiff.clone()
    dbnmeani = (-dbndiff).sum(0)
    dhprebn+= 1.0/n * (torch.ones_like(hprebn)*dbnmeani)
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
        for j in range(Xb.shape[1]):
            ix = Xb[k,j]
            dC[ix] += demb[k,j]
#     dlogits = F.softmax(logits, 1)
#     dlogits[range(n), Yb] -= 1
#     dlogits /= n
#     # 2nd layer backprop
#     dh = dlogits @ W2.T
#     dW2 = h.T @ dlogits
#     db2 = dlogits.sum(0)
#     # tanh
#     dhpreact = (1.0 - h**2) * dh
#     # batchnorm backprop
#     dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
#     dbnbias = dhpreact.sum(0, keepdim=True)
#     dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
#     # 1st layer
#     dembcat = dhprebn @ W1.T
#     dW1 = embcat.T @ dhprebn
#     db1 = dhprebn.sum(0)
#     # embedding
#     demb = dembcat.view(emb.shape)
#     dC = torch.zeros_like(C)
#     for k in range(Xb.shape[0]):
#         for j in range(Xb.shape[1]):
#             ix = Xb[k,j]
#             dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

  #   if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #     break

12297
      0/ 200000: 3.7856
  10000/ 200000: 2.1831
  20000/ 200000: 2.4150
  30000/ 200000: 2.4527
  40000/ 200000: 1.9869
  50000/ 200000: 2.3611
  60000/ 200000: 2.3825
  70000/ 200000: 2.0576
  80000/ 200000: 2.3469
  90000/ 200000: 2.1071
 100000/ 200000: 1.9602
 110000/ 200000: 2.3193
 120000/ 200000: 1.9077
 130000/ 200000: 2.4434
 140000/ 200000: 2.2627
 150000/ 200000: 2.2384
 160000/ 200000: 1.9653
 170000/ 200000: 1.8179
 180000/ 200000: 2.0022
 190000/ 200000: 1.8545


In [47]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    # measure the mean/std over the entire training set
    bnmean = hpreact.mean(0, keepdim=True)
    bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [51]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split, bngain, bnmean, bnvar, bnbias):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]
    emb = C[x] # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
    h = torch.tanh(hpreact) # (N, n_hidden)
    logits = h @ W2 + b2 # (N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

# You need to provide the BatchNorm parameters here:
split_loss('train', bngain, bnmean, bnvar, bnbias)
split_loss('val', bngain, bnmean, bnvar, bnbias)


train 2.070976972579956
val 2.1090216636657715


In [50]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
        emb = C[torch.tensor([context])] # (1,block_size,d)      
        embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
        hpreact = embcat @ W1 + b1
        hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
        h = torch.tanh(hpreact) # (N, n_hidden)
        logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    
    print(''.join(itos[i] for i in out))

mona.
mayah.
see.
mad.
ryla.
ren.
ruthadraegustered.
elin.
shi.
jen.
eden.
sana.
arleigh.
malara.
noshubergihimie.
trick.
welle.
joseus.
kuba.
geder.
